In [ ]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import numpy as np
from keras.models import load_model
from research_lib.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from weight_estimation.dataset import prepare_gtsf_data, compute_akpd_score
from weight_estimation.train import train, augment, normalize, get_data_split, train_model
from typing import Dict, Tuple


In [ ]:
df = pd.load_csv('/root/data/alok/biomass_estimation/playground/gtsf_keypoints_new.csv')

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = false;

In [ ]:
s3 = S3AccessUtils('/root/data')
akpd_scorer_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/keypoint-detection-scorer/akpd_scorer_model_TF.h5'
akpd_scorer_f, _, _ = s3.download_from_url(akpd_scorer_url)
df1 = prepare_gtsf_data('2019-03-01', '2019-09-20', akpd_scorer_f, 0.5, 1.0)

df2 = prepare_gtsf_data('2020-06-01', '2020-08-20', akpd_scorer_f, 0.5, 1.0)
df = pd.concat([df1, df2])

In [ ]:
from filter_optimization.filter_optimization_task import extract_biomass_data

gt_metadata = {'pen_id': 144,
 'gutted_average_weight': 8000,
 'gutted_weight_distribution': None,
 'expected_loss_factor': 0.16,
 'last_feeding_date': '2021-01-11',
 'harvest_date': '2021-01-15',
 'slaughter_date': '2021-01-15'}

df2 = extract_biomass_data(gt_metadata['pen_id'], '2021-01-01', '2021-01-12', 0.01)

In [ ]:
df.iloc[1000].keypoints

In [ ]:
count

In [ ]:
import numpy as np
import json
from functools import singledispatch


@singledispatch
def to_serializable(val):
    """Used by default."""
    return str(val)


@to_serializable.register(np.float32)
def ts_float32(val):
    """Used if *val* is an instance of numpy.float32."""
    return np.float64(val)

In [ ]:
json.dump(keypoints_new, open('/root/data/alok/biomass_estimation/playground/keypoints_new.json', 'w'), default=to_serializable)



In [ ]:
df['keypoints_new'] = keypoints_new

In [ ]:
df.to_csv('/root/data/alok/biomass_estimation/playground/gtsf_keypoints_new.csv')

In [ ]:
keypoints_new = []

count = 0

for index, row in df.iterrows():
    if count % 100 == 0:
        print(count, 'out of', len(df))

    count = count + 1
    
    if count < 0:
        continue
        
    oL, oR, kpL, kpR, kpLScore, kpRScore, kpLScoreAvg, kpRScoreAvg, kpLScoreMax, kpRScoreMax = get_keypoints(row)
    
    newKeypoints = {
        'leftCrop': [],
        'rightCrop': []
    }
    
    for i in np.arange(0, len(KP), 1):
        newKeypoints['leftCrop'].append({
            'xCrop': oL[i][0],
            'yCrop': oL[i][1],
            'xCropNew': kpL[i][0],
            'yCropNew': kpL[i][1],
            'xFrame': oL[i][0] + row.left_crop_metadata['x_coord'],
            'yFrame': oL[i][1] + row.left_crop_metadata['y_coord'],
            'xFrameNew': kpL[i][0] + row.left_crop_metadata['x_coord'],
            'yFrameNew': kpL[i][1] + row.left_crop_metadata['y_coord'],
            'score': kpLScore[i],
            'scoreAvg': kpLScoreAvg[i],
            'scoreMax': kpLScoreMax[i],
            'keypointType': KP[i]
        })
        
        newKeypoints['rightCrop'].append({
            'xCrop': oR[i][0],
            'yCrop': oR[i][1],
            'xCropNew': kpR[i][0],
            'yCropNew': kpR[i][1],
            'xFrame': oR[i][0] + row.right_crop_metadata['x_coord'],
            'yFrame': oR[i][1] + row.right_crop_metadata['y_coord'],
            'xFrameNew': kpR[i][0] + row.right_crop_metadata['x_coord'],
            'yFrameNew': kpR[i][1] + row.right_crop_metadata['y_coord'],
            'score': kpRScore[i],
            'scoreAvg': kpRScoreAvg[i],
            'scoreMax': kpRScoreMax[i],
            'keypointType': KP[i]
        })
        
    keypoints_new.append(newKeypoints)

In [ ]:
# config_bak_path = '/root/data/bati/model/config.json'
# config_bak = json.load(open(config_bak_path))
# config_bak

In [ ]:
config

In [ ]:
config_path = '/root/data/bati/model/production_config.json'
# config_path = '/root/data/bati/model/config_4_stage.json'

checkpoint_path = '/root/data/bati/model/model_499.pb'
config = json.load(open(config_path))

class FLAGS(object):
    input_size = tuple(config["input_size"])
    stages = config["cpm_stages"]
    batch_size = config["batch_size"]
    joints = config["num_of_joints"]
    model_path = checkpoint_path
    cmap_radius = config["center_radius"]
    keypoints_order = config["keypoints_order"]
    normalize = config["normalize"]
    heatmap_size = 512#config["heatmap_size"]
    joint_gaussian_variance = config["joint_gaussian_variance"]
    crop = config["crop"]
    augmentation = None
    
def load_pb(path_to_pb):
    with tf.io.gfile.GFile(path_to_pb, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, input_map=None,
                                return_elements=None,
                                name="",
                                op_dict=None,
                                producer_op_list=None)
        graph_nodes=[n for n in graph_def.node]
#         for t in graph_nodes:
#             print(t.name)
        return graph

mod = load_pb(checkpoint_path)

sess = tf.compat.v1.Session()
tf.compat.v1.disable_eager_execution()

sess.graph.as_default()
sess.run(tf.compat.v1.global_variables_initializer())

tf_device = '/gpu:0'
with tf.device(tf_device):
    model = mod

In [ ]:
import cv2
import urllib
import tensorflow as tf

def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

def image_resize(image, FLAGS):
    height, width, _ = image.shape
    ratio_width = width / FLAGS.input_size[0]
    ratio_height = height / FLAGS.input_size[1]
    image = cv2.resize(image, FLAGS.input_size)
    image  = image / 255.0 - 0.5
    image = image[np.newaxis, ...]
    return image

def enhance(image, clip_limit=5):
    # convert image to LAB color model
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # split the image into L, A, and B channels
    l_channel, a_channel, b_channel = cv2.split(image_lab)

    # apply CLAHE to lightness channel
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L channel with the original A and B channel
    merged_channels = cv2.merge((cl, a_channel, b_channel))

    # convert image from LAB color model back to RGB color model
    final_image = cv2.cvtColor(merged_channels, cv2.COLOR_LAB2BGR)
    return final_image 


In [ ]:
def get_keypoints(row):
    imL = row['left_image_url']
    imR = row['right_image_url']
    lco = row['left_crop_metadata']
    rco = row['right_crop_metadata']
    meta = row['camera_metadata']

    imageL = url_to_image(imL)
    imageR = url_to_image(imR)

    img1 = enhance(imageL)
    img2 = enhance(imageR)

    heightL, widthL, _ = img1.shape
    img_input = image_resize(img1, FLAGS)
    with tf.compat.v1.Session(graph=model) as sess, tf.device(tf_device):
        predict_heatmap = sess.run(config['output_name'], feed_dict = {config['input_name']: img_input})
    final_stage_heatmapL = predict_heatmap.squeeze()

    heightR, widthR, _ = img2.shape
    img_input = image_resize(img2, FLAGS)
    with tf.compat.v1.Session(graph=model) as sess, tf.device(tf_device):
        predict_heatmap = sess.run(config['output_name'], feed_dict = {config['input_name']: img_input})
    final_stage_heatmapR = predict_heatmap.squeeze()

    oL = [] # original left key-points using max method
    oR = [] # original right key-points using max method
    kpL = [] # new left key-points using avg method
    kpR = [] # new right key-points using avg method
    kpLScore = []
    kpRScore = []
    kpLScoreAvg = []
    kpRScoreAvg = []
    kpLScoreMax = []
    kpRScoreMax = []

    for c in np.arange(0, len(KP), 1):
        hm = cv2.resize(final_stage_heatmapL[..., c], (widthL, heightL))
        hm_maxL = list(np.where(hm == hm.max()))   
        
        ii = np.unravel_index(np.argsort(hm.ravel())[-10000:], hm.shape) # 2D locations corresponding to highest 10000 heatmap values
        x = np.sum(np.exp(hm[ii]) * ii[1]) / np.sum(np.exp(hm[ii]))
        y = np.sum(np.exp(hm[ii]) * ii[0]) / np.sum(np.exp(hm[ii]))

        oL.append([int(hm_maxL[1][0]), int(hm_maxL[0][0])]) 
        kpL.append([int(np.rint(x)), int(np.rint(y))])
        kpLScore.append(hm[int(np.rint(y)), int(np.rint(x))])
        kpLScoreAvg.append(np.mean(hm[ii]))
        kpLScoreMax.append(hm.max())

        hm = cv2.resize(final_stage_heatmapR[..., c], (widthR, heightR))
        hm_maxR = np.where(hm == hm.max())
    
        ii = np.unravel_index(np.argsort(hm.ravel())[-10000:], hm.shape)
        x = np.sum(np.exp(hm[ii]) * ii[1]) / np.sum(np.exp(hm[ii]))
        y = np.sum(np.exp(hm[ii]) * ii[0]) / np.sum(np.exp(hm[ii]))

        oR.append([int(hm_maxR[1][0]), int(hm_maxR[0][0])])
        kpR.append([int(np.rint(x)), int(np.rint(y))])
        kpRScore.append(hm[int(np.rint(y)), int(np.rint(x))])
        kpRScoreAvg.append(np.mean(hm[ii]))
        kpRScoreMax.append(hm.max())

    return oL, oR, kpL, kpR, kpLScore, kpRScore, kpLScoreAvg, kpRScoreAvg, kpLScoreMax, kpRScoreMax 

In [ ]:
# print(int(np.rint(y)))
# hm[np.rint(y), np.rint(x)]
get_keypoints(row)

In [ ]:
config['input_name']='input_placeholder:0'
config['output_name']='stage_3/mid_conv7/BiasAdd:0'

# for row in df.iterrows():


# def get_keypoints(row):
    # row = df.iloc[0]
imL = row['left_image_url']
imR = row['right_image_url']
lco = row['left_crop_metadata']
rco = row['right_crop_metadata']
meta = row['camera_metadata']

# imL = row['left_crop_url']
# imR = row['right_crop_url']
# lco = row['left_crop_metadata']
# rco = row['right_crop_metadata']
# meta = row['camera_metadata']

imageL = url_to_image(imL)
imageR = url_to_image(imR)

img1 = enhance(imageL)
img2 = enhance(imageR)

heightL, widthL, _ = img1.shape
img_input = image_resize(img1, FLAGS)
with tf.compat.v1.Session(graph=model) as sess, tf.device(tf_device):
    predict_heatmap = sess.run(config['output_name'], feed_dict = {config['input_name']: img_input})
final_stage_heatmapL = predict_heatmap.squeeze()

heightR, widthR, _ = img2.shape
img_input = image_resize(img2, FLAGS)
with tf.compat.v1.Session(graph=model) as sess, tf.device(tf_device):
    predict_heatmap = sess.run(config['output_name'], feed_dict = {config['input_name']: img_input})
final_stage_heatmapR = predict_heatmap.squeeze()

# SIFT matching
MIN_MATCH_COUNT = 10
GOOD_PERC = 0.7
sift = cv2.KAZE_create()
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)
good = []
for m,n in matches:
    if m.distance < GOOD_PERC*n.distance:
        good.append(m)
if len(good)>=MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
else:
    print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
    matchesMask = None

Hx=M
Hy=np.linalg.inv(M)

kpL = []
kpR = []    
kpL2R = []
kpR2L = [] 
im1ps = []
im2ps = []
gtL2R = []
gtR2L = [] 
gt1ps = []
gt2ps = []

for c in np.arange(0, len(KP), 1):
    hm = cv2.resize(final_stage_heatmapL[..., c], (widthL, heightL))

    hm_maxL = list(np.where(hm == hm.max()))   
    kpL.append([int(hm_maxL[1][0]), int(hm_maxL[0][0])]) 
    ptx=np.array([kpL[c][0],kpL[c][1],1])
    zx=np.dot(Hx,ptx)
    kpL2R.append([int(zx[0]/zx[2]), int(zx[1]/zx[2])]) 

    hm = cv2.resize(final_stage_heatmapR[..., c], (widthR, heightR))
    hm_maxR = np.where(hm == hm.max())
    kpR.append([int(hm_maxR[1][0]), int(hm_maxR[0][0])])
    pty=np.array([kpR[c][0],kpR[c][1],1])
    zy=np.dot(Hy,pty)
    kpR2L.append([int(zy[0]/zy[2]), int(zy[1]/zy[2])]) 

    im1ps.append([int((kpL[c][0]+kpR2L[c][0])/2), int((kpL[c][1]+kpR2L[c][1])/2)]) 
    im2ps.append([int((kpR[c][0]+kpL2R[c][0])/2), int((kpR[c][1]+kpL2R[c][1])/2)]) 
        
#     return kpL, kpR, im1ps, im2ps, final_stage_heatmapL, final_stage_heatmapR

In [ ]:
refined_keypoints_left = []
refined_keypoints_right = []

for c in np.arange(0, len(KP), 1):
    hm = cv2.resize(final_stage_heatmapL[..., c], (widthL, heightL))
    ii = np.unravel_index(np.argsort(hm.ravel())[-10000:], hm.shape)
    # list(np.where(hm == hm.max()))
    x = np.sum(np.exp(hm[ii]) * ii[1]) / np.sum(np.exp(hm[ii]))
    y = np.sum(np.exp(hm[ii]) * ii[0]) / np.sum(np.exp(hm[ii]))
    
    refined_keypoints_left.append([np.rint(x), np.rint(y)])

    hm = cv2.resize(final_stage_heatmapR[..., c], (widthR, heightR))
    ii = np.unravel_index(np.argsort(hm.ravel())[-10000:], hm.shape)
    # list(np.where(hm == hm.max()))
    x = np.sum(np.exp(hm[ii]) * ii[1]) / np.sum(np.exp(hm[ii]))
    y = np.sum(np.exp(hm[ii]) * ii[0]) / np.sum(np.exp(hm[ii]))
    
    refined_keypoints_right.append([np.rint(x), np.rint(y)])
    
    #ii
# X = x / hmL.shape[0] * row.left_crop_metadata['width']
# Y = y / hmL.shape[1] * row.left_crop_metadata['height']

In [ ]:
#np.exp(hm[ii])

In [ ]:
x, y

In [ ]:
refined_keypoints_right

In [ ]:
list(np.where(hm == hm.max()))

In [ ]:
x = np.sum(np.exp(hm[ii]) * ii[1]) / np.sum(np.exp(hm[ii]))
y = np.sum(np.exp(hm[ii]) * ii[0]) / np.sum(np.exp(hm[ii]))
x, y

In [ ]:
hm[112, 1174], hm[111, 1154], hm.max()

In [ ]:
for c in np.arange(0, len(KP), 1):
    hm = cv2.resize(final_stage_heatmapL[..., c], (widthL, heightL))
    print(KP[c], hm.max())

In [ ]:
kpL, kpR, im1ps, im2ps, final_stage_heatmapL, final_stage_heatmapR = get_keypoints(df2.iloc[2])

In [ ]:
im1ps

In [ ]:
kpL

In [ ]:
def display_crops(left_image_f, right_image_f, ann, overlay_keypoints=True, show_labels=False, custom_kps_left = [], custom_kps_right = []):
    fig, axes = plt.subplots(2, 1, figsize=(20, 20))
    left_image = plt.imread(left_image_f)
    right_image = plt.imread(right_image_f)
    axes[0].imshow(left_image)
    axes[1].imshow(right_image)
    left_ann, right_ann = ann['leftCrop'], ann['rightCrop']
    left_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in left_ann}
    right_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in right_ann}
    if overlay_keypoints:
        for bp, kp in left_keypoints.items():
            axes[0].scatter([kp[0]], [kp[1]], color='red', s=10)
            if show_labels:
                axes[0].annotate(bp, (kp[0], kp[1]), color='red')
        for bp, kp in custom_kps_left.items():
            axes[0].scatter([kp[0]], [kp[1]], color='cyan', s=10)
            if show_labels:
                axes[0].annotate(bp, (kp[0], kp[1]), color='cyan')
        for bp, kp in right_keypoints.items():
            axes[1].scatter([kp[0]], [kp[1]], color='red', s=10)
            if show_labels:
                axes[1].annotate(bp, (kp[0], kp[1]), color='red')
        for bp, kp in custom_kps_right.items():
            axes[1].scatter([kp[0]], [kp[1]], color='cyan', s=10)
            if show_labels:
                axes[1].annotate(bp, (kp[0], kp[1]), color='cyan')
    plt.show()

In [ ]:
import ast, os
from research.weight_estimation.keypoint_utils.optics import pixel2world
import matplotlib.pyplot as plt

s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))

left_crop_url, right_crop_url = row.left_image_url, row.right_image_url
ann, cm = row.keypoints, row.camera_metadata

# left_crop_url, right_crop_url = row.left_crop_url, row.right_crop_url
# ann, cm = row.annotation, row.camera_metadata

left_crop_f, _, _ = s3.download_from_url(left_crop_url)
right_crop_f, _, _ = s3.download_from_url(right_crop_url)
wkps1 = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)

In [ ]:
oDists = {}
nDists = {}

for kp in KP:
    oDists[kp] = []
    nDists[kp] = []

cnt = 0
stop = False
    
for index, row in df.iterrows():
    if stop:
        continue
    for i in np.arange(0, len(KP)):
        newKeypointLeft = [kp for kp in keypoints_new[cnt]['leftCrop'] if kp['keypointType'] == KP[i]][0]
        newKeypointRight = [kp for kp in keypoints_new[cnt]['rightCrop'] if kp['keypointType'] == KP[i]][0]
    
        gtKeypoint = [kp for kp in row.keypoints['leftCrop'] if kp['keypointType'] == KP[i]][0]
        gtX = gtKeypoint['xCrop']
        gtY = gtKeypoint['yCrop']

        oX = newKeypointLeft['xCrop']
        oY = newKeypointLeft['yCrop']

        nX = newKeypointLeft['xCropNew']
        nY = newKeypointLeft['yCropNew']
        
        oDist = np.abs(gtX - oX) + np.abs(gtY - oY)
        nDist = np.abs(gtX - nX) + np.abs(gtY - nY)
        
        if oDist > 1000:
            print(KP[i], index, gtX, oX, gtY, oY)
            stop = True

        oDists[KP[i]].append(oDist)
        nDists[KP[i]].append(nDist)

        gtKeypoint = [kp for kp in row.keypoints['rightCrop'] if kp['keypointType'] == KP[i]][0]
        gtX = gtKeypoint['xCrop']
        gtY = gtKeypoint['yCrop']

        oX = newKeypointRight['xCrop']
        oY = newKeypointRight['yCrop']

        nX = newKeypointRight['xCropNew']
        nY = newKeypointRight['yCropNew']

        oDist = np.abs(gtX - oX) + np.abs(gtY - oY)
        nDist = np.abs(gtX - nX) + np.abs(gtY - nY)

        oDists[KP[i]].append(oDist)
        nDists[KP[i]].append(nDist)

    cnt = cnt + 1
    
#     if cnt >= 449:
#         break
#     np.mean(oDists), np.mean(nDists)